Full rate estimation ad additional systematics

In [1]:
card_name = 'v5'

In [2]:
fit_real_data = False

In [3]:
category = 'high'

# Import

In [4]:
import sys, os, pickle
from glob import glob
sys.path.append('../lib')
sys.path.append('../analysis')
from categoriesDef import categories
import itertools
import json, yaml
from IPython.display import IFrame, Image, display
from multiprocessing import Pool

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from array import array

import uproot as ur
import ROOT as rt
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)
import root_numpy as rtnp

from analysis_utilities import drawOnCMSCanvas, DSetLoader
from pT_calibration_reader import pTCalReader
from histo_utilities import create_TH1D, create_TH2D, std_color_list
from cebefo_style import Set_2D_colz_graphics
from gridVarQ2Plot import plot_gridVarQ2
from progressBar import ProgressBar
from lumi_utilities import getLumiByTrigger

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1

if fit_real_data:
    CMS_lumi.extraText = "Preliminary"
else:
    CMS_lumi.extraText = "Simulation Preliminary"

donotdelete = []

Welcome to JupyROOT 6.12/07


In [6]:
cat = categories[category]

# Define binning

In [7]:
binning = {
    'q2'      : array('d', [-2, 1.5, 4, 6, 12]),
    'M2_miss' : [
        array('d', [-2.5] + list(np.arange(-1.8, 0.8, 0.4)) + [6] ),
        array('d', [-2.5] + list(np.arange(-1.8, 1.6, 0.4)) + [6] ),
        array('d', [-2.5] + list(np.arange(-1.8, 3.0, 0.4)) + [6] ),
        array('d', [-2.5] + list(np.arange(-1.8, 5.2, 0.4)) + [6] ),
    ],
    'Est_mu'  : [
        array('d', [0.5] + list(np.arange(0.8, 2.3, 0.1)) + [2.5] ),
        array('d', [0.5] + list(np.arange(0.8, 2.5, 0.1)) + [2.5] ),
        [20, 0.50, 2.500],
        [20, 0.50, 2.500],
    ]
}

# Load the datasets

In [8]:
MCsample = {
'mu' : DSetLoader('B0_MuNuDmst_PU20'),
'tau' : DSetLoader('B0_TauNuDmst_PU20'),
'Hc' : DSetLoader('B0_DmstHc_PU20'),
'Dstst' : DSetLoader('Bp_MuNuDstst_PU20')
}
dSet = {}
for n, s in MCsample.iteritems():
    dSet[n] = rtnp.root2array(s.skimmed_dir + '/{}.root'.format(cat.name))

283859
22431
6876
6884


In [9]:
if fit_real_data:
    creation_date = '200304'
    locRD = '../data/cmsRD/skimmed/B2DstMu_{}_{}.root'.format(creation_date, cat.name)
    dSet['data'] = rtnp.root2array(locRD)
    dataDir = '../data/cmsRD'
    datasets_loc = glob(dataDir + '/ParkingBPH*/*RDntuplizer_B2DstMu_{}_CAND.root'.format(creation_date))
    lumi_tot = getLumiByTrigger(datasets_loc, cat.trg, verbose=True)
    CMS_lumi.integrated_lumi = lumi_tot
else:
    lumi_tot = 25 #fb^-1

# Load all the calibrations

In [10]:
r = np.zeros((3,2))
for i, fn in enumerate(glob('../data/calibration/totalRate/ratioB02JPsiKst_*.txt')):
    with open(fn, 'r') as faux:
        aux = faux.readlines()[0][:-1].split(' ')
        r[i] = [float(aux[0]), float(aux[1])]
s2 = np.square(r[:,1])
num = np.sum(r[:,0]/s2)
den = np.sum(1./s2)
RDoMC_normRatio = [num/den, np.sqrt(1/den)]
print 'Expected ratio between RD and MC norm: {:.3f} +/- {:.3f}'.format(RDoMC_normRatio[0], RDoMC_normRatio[1])

Expected ratio between RD and MC norm: 2.632 +/- 0.063


In [11]:
decayBR = pickle.load(open('../data/forcedDecayChannelsFactors.pickle', 'rb'))

In [13]:
loc = '../data/calibration/triggerScaleFactors/'
fTriggerSF = rt.TFile.Open(loc + 'HLT_' + cat.trg + '_SF.root', 'READ')
hTriggerSF = fTriggerSF.Get('hSF_HLT_' + cat.trg)

In [ ]:
def computeTrgSF():
    pass

In [14]:
fMuonIDSF = rt.TFile.Open('../data/calibration/muonIDscaleFactors/Run2018ABCD_SF_MuonID_Jpsi.root', 'READ')
hMuonIDSF = fMuonIDSF.Get('NUM_SoftID_DEN_genTracks_pt_abseta')

In [12]:
cal_pT = pTCalReader(calibration_file='../data/calibration/B0pTspectrum/pwWeights_{}.txt'.format(cat.name))

# Create MC histograms

In [20]:
histo = {}
for n, ds in dSet.iteritems():
    print '\n'
    print '----------->', n, '<-------------'
    sMC = MCsample[n]
    
    nTotSelected = ds['q2'].shape[0]
    print 'N tot selected: {:.1f}k'.format(1e-3*nTotSelected)
    nGenExp = sMC.effMCgen['xsec'][0] * lumi_tot * RDoMC_normRatio[0] #uncertainty will be set in fit
    eff = [1, 0]
    for f, df in [sMC.effMCgen['effGEN'], decayBR[n], sMC.effCand['effCAND'], sMC.getSkimEff(cat.name)]:
        eff[0] *= f
        eff[1] += np.square(df/f)
    eff[1] = eff[0] * np.sqrt(eff[1])
    print 'N tot expected: {:.1f}k'.format(1e-3*nGenExp*eff[0])
    
    wVar = {}
    
    trgSF = np.ones(nTotSelected)
    trgSFUnc = np.ones(nTotSelected)
    x = np.column_stack((ds['trgMu_pt'], ds['trgMu_eta'], ds['trgMu_sigdxy'])
    for i, (pt, eta, ip) in enumerate(x):
        ix = hTriggerSF.GetXaxis().FindBin(pt)
        iy = hTriggerSF.GetYaxis().FindBin(ip)
        iz = hTriggerSF.GetZaxis().FindBin(np.abs(eta))
        trgSF[i] = hTriggerSF.GetBinContent(ix, iy, iz)
        trgSFUnc[i] = hTriggerSF.GetBin(ix, iy, iz)
        trgSFUnc[i] = hTriggerSF.GetBinError(trgSF[i] + hTriggerSF.GetBinError(ib))
    # Divide them for the weight so later you can simply multiply back to get the value
    wVar['trgSFUp'] = 1 + trgSFUnc[i]/trgSF
    wVar['trgSFDown'] = 1 - trgSFUnc[i]/trgSF
    
    
    auxArrMC = rtnp.tree2array(tMC_skimmed, branches=['MC_mup_pt', 'MC_mup_eta', 'MC_mum_pt', 'MC_mum_eta'])
muonSF = np.ones(auxArrMC.shape[0])
for i, (ptp, etap, ptm, etam) in enumerate(auxArrMC):
ix = hMuonIDSF.GetXaxis().FindBin(ptp)
iy = hMuonIDSF.GetYaxis().FindBin(np.abs(etap))
wp = hMuonIDSF.GetBinContent(ix, iy)
ix = hMuonIDSF.GetXaxis().FindBin(ptm)
iy = hMuonIDSF.GetYaxis().FindBin(np.abs(etam))
wm = hMuonIDSF.GetBinContent(ix, iy)
muonSF[i] = wp * wm



-----------> mu <-------------
N tot selected: 283.9k
N tot expected: 404.3k


-----------> tau <-------------
N tot selected: 22.4k
N tot expected: 11.8k


-----------> Hc <-------------
N tot selected: 6.9k
N tot expected: 2.0k


-----------> Dstst <-------------
N tot selected: 6.9k
N tot expected: 6.8k


In [21]:
ds['trgMu_pt', 'trgMu_eta', 'trgMu_sigdxy']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
histo = {}
for i_q2 in range(len(binning['q2'])-1):
    q2_l = binning['q2'][i_q2]
    q2_h = binning['q2'][i_q2 + 1]

    for var in ['M2_miss', 'Est_mu']:
        cat_name = var+'_q2bin'+str(i_q2)
        histo[cat_name] = {}
        for k, d in dSet.iteritems():          
            q2_bin = np.logical_and(d['q2'] > q2_l, d['q2'] < q2_h)
            
            if k in ['mu', 'tau']:
                for k_wpT in ['C', 'Up', 'Down']:
                    pFF_list = ['Central']
                    if k_wpT == 'C':
                        pFF_list += ['R0', 'R1', 'R2', 'RhoSq']
                    for n_pFF in pFF_list:
                        var_pFF_list = [''] if n_pFF == 'Central' else ['Up', 'Down']
                        for var_pFF in var_pFF_list:
                            h_name = k
                            if k_wpT != 'C':
                                h_name += '__B0pT' + k_wpT
                            elif n_pFF != 'Central':
                                h_name += '__B2DstCLN' + n_pFF + var_pFF

                            w = cal_pT.f[k_wpT](d['MC_B_pt'])
                            w *= d['wh_CLN'+n_pFF+var_pFF]
                            norm = np.sum(w)
                            w = w[q2_bin]
                            
                            h = create_TH1D(d[var][q2_bin], name=h_name, title=h_name, 
                                            binning=binning[var][i_q2], 
                                            opt='underflow,overflow',
                                            weights=w,
                                            scale_histo=1./norm,
                                           )

                            histo[cat_name][h_name] = h
            elif k in ['Hc']:
                for k_wpT in ['C', 'Up', 'Down']:
                    h_name = k
                    if k_wpT != 'C':
                        h_name += '__B0pT' + k_wpT
                        
                    w = cal_pT.f[k_wpT](d['MC_B_pt'][q2_bin])
                    
                    norm = np.sum(cal_pT.f[k_wpT](d['MC_B_pt']))
                    h = create_TH1D(d[var][q2_bin], name=h_name, title=h_name, 
                                    binning=binning[var][i_q2], 
                                    opt='underflow,overflow',
                                    weights=w,
                                    scale_histo=1./norm,
                                   )
                    
                    histo[cat_name][h_name] = h
            elif k in ['Dstst']:
                norm = float(d[var].shape[0])
                h = create_TH1D(d[var][q2_bin], name=k, title=k, 
                                binning=binning[var][i_q2], 
                                opt='underflow,overflow',
                                scale_histo=1./norm,
                               )
                histo[cat_name][k] = h

Generate Pseudo-data

In [ ]:
expected_evts = {}
for n in dSet.keys():
    if n == 'data': continue
    print '--->', n
    xsec_eff = dic_MCeff[n]['xsec']
    print 'Pythia xsec: {:1.2e} pb'.format(xsec_eff)
    xsec_eff *= dic_decayBR[n][0]
    print 'Forced decays BR: {:1.2e}'.format(dic_decayBR[n][0])
    xsec_eff *= dic_MCeff[n]['CMSSWFilterEff']
    print'Eff CMSSW filter: {:1.2e}'.format(dic_MCeff[n]['CMSSWFilterEff'])
    xsec_eff *= dic_MCeff[n]['ntupplizerEff']
    print'Eff ntuplizer: {:1.2e}'.format(dic_MCeff[n]['ntupplizerEff'])
    xsec_eff *= dic_MCeff[n]['analysisSelEff']
    print'Eff selection: {:1.2e}'.format(dic_MCeff[n]['analysisSelEff'])
    xsec_eff *= 1e3 # pb -> fb
    print '\nExpected evts/fb: {:.0f}'.format(xsec_eff)
    print '\n'
    expected_evts[n] = xsec_eff

# Create (pseudo-)data histogram

In [ ]:
for i_q2 in range(len(binning['q2'])-1):
    q2_l = binning['q2'][i_q2]
    q2_h = binning['q2'][i_q2 + 1]

    for var in ['M2_miss', 'Est_mu']:
        cat_name = var+'_q2bin'+str(i_q2)
        histo[cat_name] = {}
        for k, d in dSet.iteritems():          
            q2_bin = np.logical_and(d['q2'] > q2_l, d['q2'] < q2_h)
            
            if k == 'data':
                histo[cat_name][k] = create_TH1D(d[var][q2_bin], 
                                                 name='data_obs', title='Data Obs',
                                                 binning=binning[var][i_q2],
                                                 opt='underflow,overflow'
                                                )

In [ ]:
SM_RDst = 0.26
rawR_exp = SM_RDst*expected_evts['tau']/expected_evts['mu']
print 'Raw ratio: {:1.2e}'.format(rawR_exp)
f_Hc = expected_evts['Hc']/np.sum(expected_evts.values())
print 'f_Hc: {:1.2e}'.format(f_Hc)
f_Dstst = expected_evts['Dstst']/np.sum(expected_evts.values())
print 'f_Dstst: {:1.2e}'.format(f_Dstst)

f_bkg = f_Hc + f_Dstst
if fit_real_data:
    from lumi_utilities import getLumiReport
    lumi_tot, lumi_dic = getLumiReport(glob(file_loc['data']))
else:    
    lumi_tot = 10. #fb-1
    if trgMu_pt_region == 'high':
        N_exp_data = 98500
    elif trgMu_pt_region == 'mid':
        N_exp_data = 47000
    N_exp = N_exp_data*lumi_tot/5.25
    N_B2mu_inj = N_exp*(1-f_bkg)/(1+rawR_exp)

    n_mu = np.random.poisson(lam=N_B2mu_inj)
    w_mu = []
    sum_w_mu = 0
    idx_mu = []
    print('Generating {:.1f}k pseudo-events of mu'.format(n_mu*1.0e-3))
    pb = ProgressBar(n_mu)
    while sum_w_mu < n_mu:
        pb.show(np.ceil(sum_w_mu))
        i = np.random.randint(0, dSet['mu']['q2'].shape[0], size=(1, ))[0]
        w = cal_pT.f['C'](dSet['mu']['MC_B_pt'][i])
        w *= dSet['mu']['wh_CLNCentral'][i]
        idx_mu.append(i)
        w_mu.append(w)
        sum_w_mu += w
    pb.show(n_mu-1)

    n_tau = np.random.poisson(lam=N_B2mu_inj*rawR_exp)
    w_tau = []
    idx_tau = []
    print('Generating {:.1f}k pseudo-events of tau'.format(n_tau*1.0e-3))
    pb = ProgressBar(n_tau)
    pb.show(0)
    while np.sum(w_tau) < n_tau:
        pb.show(np.sum(w_tau))
        i = np.random.randint(0, dSet['tau']['q2'].shape[0], size=(1, ))[0]
        w = cal_pT.f['C'](dSet['tau']['MC_B_pt'][i])
        w *= dSet['tau']['wh_CLNCentral'][i]
        idx_tau.append(i)
        w_tau.append(w)
    pb.show(n_tau-1)
    
    n_Hc = np.random.poisson(lam=N_exp*f_Hc)
    w_Hc = []
    idx_Hc = []
    print('Generating {:.1f}k pseudo-events of Hc'.format(n_Hc*1.0e-3))
    pb = ProgressBar(n_Hc)
    while np.sum(w_Hc) < n_Hc:
        pb.show(np.sum(w_Hc))
        i = np.random.randint(0, dSet['Hc']['q2'].shape[0], size=(1, ))[0]
        w = cal_pT.f['C'](dSet['Hc']['MC_B_pt'][i])
        idx_Hc.append(i)
        w_Hc.append(w)
    pb.show(n_Hc-1)
    
    n_Dstst = np.random.poisson(lam=N_exp*f_Dstst)
    print('Generating {:.1f}k pseudo-events of Dstst'.format(n_Dstst*1.0e-3))
    idx_Dstst = np.random.randint(0, dSet['Dstst']['q2'].shape[0], size=(n_Dstst, ))

    pseudo_data = {}
    pseudo_w = np.array(w_mu + w_tau + w_Hc + [1.]*len(idx_Dstst))
    for var in binning.keys():
        pseudo_data[var] = np.concatenate((dSet['mu'][var][idx_mu], 
                                           dSet['tau'][var][idx_tau],
                                           dSet['Hc'][var][idx_Hc],
                                           dSet['Dstst'][var][idx_Dstst]
                                          ))

    for i_q2 in range(len(binning['q2'])-1):
        q2_l = binning['q2'][i_q2]
        q2_h = binning['q2'][i_q2 + 1]

        for var in ['M2_miss', 'Est_mu']:
            cat_name = var+'_q2bin'+str(i_q2)

            sel = np.logical_and(pseudo_data['q2'] > q2_l, pseudo_data['q2'] < q2_h)
            
            h = create_TH1D(pseudo_data[var][sel], 
                            name='data_obs', title='data obs',
                            binning=binning[var][i_q2],
                            opt='underflow,overflow',
                            weights=pseudo_w[sel]
                           )
            h.Sumw2(0)
            for i in range(1, h.GetNbinsX()+1):
                h.SetBinContent(i, round(h.GetBinContent(i)))
            h.Sumw2()
            histo[cat_name]['data'] = h


    print 'r_toy = {:.2f}%'.format(100.*n_tau/n_mu)

In [ ]:
N_data = 0
N_mu = 0
N_tau = 0
N_Hc = 0
N_Dstst = 0
for k, h_dic in histo.iteritems():
    if 'Est_mu' in k:
        N_data += h_dic['data'].Integral()
        N_mu += h_dic['mu'].Integral()
        N_tau += h_dic['tau'].Integral()
        N_Hc += h_dic['Hc'].Integral()
        N_Dstst += h_dic['Dstst'].Integral()
N_B2mu_exp = N_data*(1-f_bkg)/(1+rawR_exp)

print 'Number of data events: {:.0f}'.format(N_data)
print 'Number of B0 -> D*munu expected: {:.0f}'.format(N_B2mu_exp)
print 'Total norm'
print 'Mu: {:.3f}'.format(N_mu)
print 'Tau: {:.3f}'.format(N_tau)
print 'Hc: {:.3f}'.format(N_Hc)
print 'Dstst: {:.3f}'.format(N_Dstst)

In [ ]:
histo_file_dir = '../data/_root/histos4combine/'
if not os.path.isdir(histo_file_dir):
    os.makedirs(histo_file_dir)
histo_file_loc = {}
for cat_name, h_dic in histo.iteritems():
    histo_file_loc[cat_name] = histo_file_dir+'{}_{}.root'.format(card_name, cat_name)
    tf = rt.TFile(histo_file_loc[cat_name], 'recreate')
    for v in h_dic.values():
        v.Write()
    tf.Close()

In [ ]:
CMS_lumi.integrated_lumi = lumi_tot
scale_dic = {
    'tau': N_B2mu_exp*rawR_exp, 
    'mu' : N_B2mu_exp,
    'Hc' : N_data*f_Hc,
    'Dstst' : N_data*f_Dstst
            }

c = plot_gridVarQ2(CMS_lumi, binning, histo, scale_dic, min_y=1, logy=True)

# CMS_lumi.integrated_lumi = ''
# histNorm = {}
# for k, hdic in histo.iteritems():
#     if not k in histNorm.keys():
#         histNorm[k] = {}
#     for kk, h in hdic.iteritems():
#         hAux = h.Clone()
#         hAux.Scale(1./float(N_data))
#         histNorm[k][kk] = hAux

# c = plot_gridVarQ2s(CMS_lumi, binning, histNorm, scale_dic, min_y=1, logy=False, iPad_legend=7)

# Write the card

In [ ]:
sig_processes = ['tau', 'mu']
bkg_processes = ['Hc', 'Dstst']
processes = sig_processes + bkg_processes
categories = np.sort([k for k in histo.keys() if not '__' in k])

In [ ]:
card_location = 'cards/{}.txt'.format(card_name)
fc = open(card_location, 'w')

In [ ]:
# number of different categories
card = 'imax *\n'
# number of processes minus one
card += 'jmax {}\n'.format(len(processes)-1)
# number of nuissance parameters
card += 'kmax *\n'
card += '--------------------------------------------------------------\n'

# shape file location
for k in categories:
    card += 'shapes * {} {} $PROCESS $PROCESS__$SYSTEMATIC\n'.format(k, histo_file_loc[k])
card += '--------------------------------------------------------------\n'

# number of events observed
card += 'bin ' + ' '.join(categories) + '\n'
obs = map(lambda k: '{:.0f}'.format(histo[k]['data'].Integral()), categories)
obs = ' '.join(obs)
card += 'observation ' + obs + '\n'
card += '--------------------------------------------------------------\n'


# MC expected events
aux_bin = ''
aux_proc_name = ''
aux_proc_id = ''
aux_proc_rate = ''
for c, p in itertools.product(categories, processes):
    aux_bin += ' '+c
    aux_proc_name += ' '+p
    aux_proc_id += ' '+str(np.argmax(np.array(processes) == p))
    aux_proc_rate += ' {:.4f}'.format(histo[c][p].Integral())
    
card += 'bin' + aux_bin + '\n'
card += 'process' + aux_proc_name + '\n'
# Zero or negative for sig and positive for bkg
card += 'process' + aux_proc_id + '\n'
# Expected rate
card += 'rate' + aux_proc_rate + '\n'
card += '--------------------------------------------------------------\n'

Add additional rate parameters

In [ ]:
card += 'N_B2mu rateParam * tau {:.1f} [0,{:.0f}]\n'.format(scale_dic['mu'], 1.2*N_data)
card += 'N_B2mu rateParam * mu {:.1f} [0,{:.0f}]\n'.format(scale_dic['mu'], 1.2*N_data)
card += 'N_B2DstHc rateParam * Hc {:.1f} [0,{:.0f}]\n'.format(scale_dic['Hc'], 1.2*N_data)
card += 'N_B2Dstst rateParam * Dstst {:.1f} [0,{:.0f}]\n'.format(scale_dic['Dstst'], 1.2*N_data)
card += '--------------------------------------------------------------\n'

Systematics

In [ ]:
aux_B0pT = ''
for c, p in itertools.product(categories, processes):
    if p in ['tau', 'mu', 'Hc']:
        aux_B0pT += ' 1.'
    else:
        aux_B0pT += ' -'
card += 'B0pT shape' + aux_B0pT + '\n'

In [ ]:
for n_pFF in ['R0', 'R1', 'R2', 'RhoSq']:
    aux = ''
    for c, p in itertools.product(categories, processes):
        if p in ['tau', 'mu']:
            aux += ' 1.'
        else:
            aux += ' -'
    card += 'B2DstCLN{} shape'.format(n_pFF) + aux + '\n'

MC statistic systematics

In [ ]:
card += '* autoMCStats 0 1 1\n'
card += '--------------------------------------------------------------\n'

Defining groups of systematics

In [ ]:
# autoMCStats group = defined by default when using autoMCStats
card += 'B2DstFF group = B2DstCLN' + ' B2DstCLN'.join(['R0', 'R1', 'R2', 'RhoSq']) + '\n'
card += 'bkgMC_norm group = N_B2DstHc N_B2Dstst\n'
card += 'all_card group = N_B2mu N_B2DstHc N_B2Dstst B0pT\n'

In [ ]:
print card

In [ ]:
fc.write(card)
fc.close()

### Create output directory

In [ ]:
outdir = 'results/' + card_name

if os.path.isdir(outdir):
    os.system('rm -rf ' + outdir)
os.system('mkdir ' + outdir);

# Run Combine

## Create the workspace

In [ ]:
cmd = 'text2workspace.py ' + card_location 
cmd += ' -o ' + card_location.replace('.txt', '.root')
cmd += ' --no-b-only'
cmd += ' --verbose 1'
# cmd += ' --no-wrappers'
os.system(cmd)

### Run the Maximum Likelyhood fit

In [ ]:
cmd = 'combine'
cmd += ' -M FitDiagnostics'
cmd += ' --robustFit 1'
cmd += ' --cminDefaultMinimizerStrategy 0'
cmd += ' --skipBOnlyFit'
cmd += ' -d ' + card_location.replace('.txt', '.root')
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --setParameters r={:.2f}'.format(rawR_exp)
cmd += ' --setParameterRanges r=0.001,1'
cmd += ' --trackParameters N_B2mu,N_B2DstHc,N_B2Dstst'
cmd += ' -n {}'.format(card_name)
cmd += ' --out ' + outdir
cmd += ' --saveShapes --saveWithUncertainties'
cmd += ' --plots'
cmd += ' --verbose 1'

print cmd
os.system(cmd)
os.system('mv combine_logger.out ' + outdir + '/')
os.system('mv ./*.root ' + outdir + '/')

In [ ]:
cmd = 'python diffNuisances.py '.format(os.environ['CMSSW_BASE'])
cmd += glob(outdir + '/fitDiagnostics{}.root'.format(card_name))[0]
cmd += ' --skipFitB'
cmd += ' -g {}/nuisance_pull.root'.format(outdir)
print cmd
os.system(cmd)

In [ ]:
import tdrstyle
tdrstyle.setTDRStyle()

def plot_gridVarQ2s(CMS_lumi, binning, histo, scale_dic, 
                    min_y=1e-4, 
                    draw_pulls=False, 
                    pulls_ylim=[0.8, 1.2], 
                    logy=False, iPad_legend=1):
    col_dic = {'mu': rt.kAzure+1, 'tau': rt.kRed-4, 'Hc':rt.kGreen+1, 'Dstst': rt.kViolet-7}
    
    canvas = rt.TCanvas('c_out', 'c_out', 50, 50, 2*600, 400*len(binning['q2'])-1)
    canvas.SetTickx(0)
    canvas.SetTicky(0)
    canvas.Divide(2, len(binning['q2'])-1, 0.001, 0.001)
    
    canvas.dnd = []

    vars_to_plot = ['M2_miss', 'Est_mu']

    xAx_title = {'M2_miss':'m^{2}_{miss} [GeV^{2}]', 'Est_mu':'E_{#mu}* [GeV]'}
    label_dic = {'data' : 'Data',
                 'mu'   : 'B#rightarrow D*#mu#nu',
                 'tau'  : 'B#rightarrow D*#tau#nu',
                 'Hc'   : 'B#rightarrow D*H_{c}',
                 'Dstst': 'B#rightarrow D**#mu#nu'
                }
    
    rt.TGaxis.SetMaxDigits(3)

    max_entries = dict(zip(vars_to_plot, [0]*len(vars_to_plot)))
    for k, h_dic in histo.iteritems():
        if 'M2' in k:
            max_entries['M2_miss'] = max(h_dic['data'].GetMaximum(), max_entries['M2_miss'])
        elif 'Est' in k:
            max_entries['Est_mu'] = max(h_dic['data'].GetMaximum(), max_entries['Est_mu'])
    max_entries['M2_miss'] = 0.09*0.55
    max_entries['Est_mu'] = 0.035*0.4

    for i_q2 in range(len(binning['q2'])-1):
        q2_l = binning['q2'][i_q2]
        q2_h = binning['q2'][i_q2 + 1]
        q2_txt = '{:.1f} <  q^{{2}}  < {:.1f} GeV^{{2}}'.format(q2_l, q2_h)

        for i_v, vark in enumerate(vars_to_plot):            
            cat_name = vark+'_q2bin'+str(i_q2)
            h_dic = histo[cat_name]
            
            i_pad = i_q2*2 + i_v + 1
            pad_master = canvas.cd(i_pad)
            
            if not draw_pulls:
                pad = rt.TPad('pmain_'+cat_name, 'pmain_'+cat_name, 0, 0, 1, 1)
                pad.SetBottomMargin(0.2)
            else:
                pad = rt.TPad('pmain_'+cat_name, 'pmain_'+cat_name, 0, 0.25, 1, 1)
                pad.SetBottomMargin(0.)
            pad.SetTopMargin(0.07)
            pad.SetRightMargin(0.05)
            pad.SetLeftMargin(0.18)
            pad.Draw()
            pad.cd()

            h = h_dic['data'].Clone('h_aux_data_'+cat_name)
            if 'data' in scale_dic.keys(): h.Scale(scale_dic['data'])
            h.SetLineColor(1)
            h.SetLineWidth(1)
            h.SetMarkerColor(1)
            h.SetMarkerStyle(20)
            h.GetXaxis().SetTitle(xAx_title[vark])
            h.GetXaxis().SetTitleSize(0.075)
            h.GetXaxis().SetLabelSize(0.07)
            h.GetYaxis().SetTitleOffset(1.5)
            h.GetXaxis().SetTitleOffset(1.1)
            h.GetYaxis().SetTitleSize(0.06)
            h.GetYaxis().SetLabelSize(0.07)
            iunits = xAx_title[vark].find('[') + 1
            h.GetYaxis().SetTitle('a.u. / {:.2f} '.format(h.GetBinWidth(3)) + xAx_title[vark][iunits:-1])
            max_y = max_entries[vark]
            if 'data' in scale_dic.keys(): 
                max_y *= scale_dic['data']
            h.GetYaxis().SetRangeUser(min_y, max_y)
            
            
            h_Dstst = h_dic['Dstst'].Clone('h_aux_Dstst_'+cat_name)
            if 'Dstst' in scale_dic.keys(): h_Dstst.Scale(scale_dic['Dstst'])
            h_Dstst.SetLineWidth(0)
            h_Dstst.SetFillColor(col_dic['Dstst'])
            h_Dstst.SetFillStyle(1)
            h_Dstst.Sumw2(0)
            
            h_Hc = h_dic['Hc'].Clone('h_aux_Hc_'+cat_name)
            if 'Hc' in scale_dic.keys(): h_Hc.Scale(scale_dic['Hc'])
            h_Hc.Add(h_Dstst)
            h_Hc.SetLineWidth(0)
            h_Hc.SetFillColor(col_dic['Hc'])
            h_Hc.SetFillStyle(1)
            h_Hc.Sumw2(0)
            
            h_tau = h_dic['tau'].Clone('h_aux_tau_'+cat_name)
            if 'tau' in scale_dic.keys(): h_tau.Scale(scale_dic['tau'])
            h_tau.Add(h_Hc)
            h_tau.SetLineWidth(0)
            h_tau.SetFillColor(col_dic['tau'])
            h_tau.SetFillStyle(1)
            h_tau.Sumw2(0)
            
            h_mu = h_dic['mu'].Clone('h_aux_mu_'+cat_name)
            if 'mu' in scale_dic.keys(): h_mu.Scale(scale_dic['mu'])
            h_mu.Add(h_tau)
            h_mu.SetLineWidth(0)
            h_mu.SetFillColor(col_dic['mu'])
            h_mu.SetFillStyle(1)
            h_mu.Sumw2(0)
            
            
            h.Draw('E1')
            h_mu.DrawCopy('SAME')
            h_tau.Draw('SAME')
            h_Hc.Draw('SAME')
            h_Dstst.Draw('SAME')
            h.Draw('SAMEE1') #Draw it a second time to bring it in foreground

            l = rt.TLatex()
            l.SetTextAlign(11)
            l.SetTextSize(0.06)
            l.SetTextFont(42)
#             l.DrawLatexNDC(0.22, 0.85, q2_txt)

            CMS_lumi.CMS_lumi(pad, -1, 33, cmsTextSize=0.75*1.2, lumiTextSize=0.6*1.2)
            if logy:
                pad.SetLogy()

            if i_pad == iPad_legend:
                leg = rt.TLegend(0.6, 0.3, 0.9, 0.7)
                leg.SetTextFont(42)
                leg.SetTextAlign(12)
                leg.SetLineWidth(0)
                leg.SetBorderSize(0)
                leg.AddEntry(h, label_dic['data'], 'lep')
                leg.AddEntry(h_mu, label_dic['mu'], 'f')
                leg.AddEntry(h_tau, label_dic['tau'], 'f')
                leg.AddEntry(h_Hc, label_dic['Hc'], 'f')
                leg.AddEntry(h_Dstst, label_dic['Dstst'], 'f')
                leg.Draw()
                canvas.dnd.append(leg)
                
            canvas.dnd.append([pad, h, h_tau, h_mu, h_Hc, h_Dstst])    

            if draw_pulls:
                pad_master.cd()

                pad = rt.TPad('ppull_'+cat_name, 'ppull_'+cat_name, 0, 0, 1, 0.25)
                pad.SetBottomMargin(0.5)
                pad.SetTopMargin(0)
                pad.SetRightMargin(0.05)
                pad.SetLeftMargin(0.18)
                pad.Draw()
                pad.cd()
                
                h_dr = h.Clone('h_aux_dataratio_'+cat_name)
                h_dr.GetYaxis().SetTitle('RD/MC')
                h_tot = h_dic['total'].Clone('h_aux_total_'+cat_name)
                g_up = rt.TGraph()
                g_up.SetPoint(0, h_tot.GetBinCenter(1)-0.5*h_tot.GetBinWidth(1), 1)
                g_down = rt.TGraph()
                g_down.SetPoint(0, h_tot.GetBinCenter(1)-0.5*h_tot.GetBinWidth(1), 1)
                for i in range(1, h_dr.GetNbinsX()+1):
                    c = h_dr.GetBinContent(i)
                    e = h_dr.GetBinError(i)
                    c_MC = h_tot.GetBinContent(i)
                    e_MC = h_tot.GetBinError(i)
                    h_dr.SetBinContent(i, c/c_MC)
                    h_dr.SetBinError(i, e/c_MC)
                    x_low = h_tot.GetBinCenter(i) - 0.5*h.GetBinWidth(i)
                    x_up = h_tot.GetBinCenter(i) + 0.5*h.GetBinWidth(i)
                    g_up.SetPoint(2*i-1, x_low, (c_MC+e_MC)/c_MC)
                    g_up.SetPoint(2*i, x_up, (c_MC+e_MC)/c_MC)
                    g_down.SetPoint(2*i-1, x_low, (c_MC-e_MC)/c_MC)
                    g_down.SetPoint(2*i, x_up, (c_MC-e_MC)/c_MC)
                g_up.SetPoint(2*i+1, x_up, 1)
                g_down.SetPoint(2*i+1, x_up, 1)
                
                h_dr.GetYaxis().SetRangeUser(pulls_ylim[0], pulls_ylim[1])
                h_dr.GetYaxis().SetTitleOffset(0.35)
                h_dr.GetYaxis().SetTitleSize(0.2)
                h_dr.GetYaxis().SetLabelSize(0.2)
                h_dr.GetYaxis().SetNdivisions(402)
                h_dr.GetXaxis().SetTitleOffset(0.95)
                h_dr.GetXaxis().SetTitleSize(0.22)
                h_dr.GetXaxis().SetLabelSize(0.22)
                h_dr.GetXaxis().SetTickSize(0.07)
                
                h_dr.Draw('E1')
                
                g_up.SetFillColor(rt.kGray)
                g_up.SetFillStyle(1)
                g_up.Draw('F')
                g_down.SetFillColor(rt.kGray)
                g_down.SetFillStyle(1)
                g_down.Draw('F')
                l = rt.TLine()
                l.SetLineColor(rt.kGray+1)
                l.SetLineWidth(1)
                l.SetLineStyle(9)
                x_low = h_tot.GetBinCenter(1)-0.5*h.GetBinWidth(1)
                x_high = h_tot.GetBinCenter(i)+0.5*h.GetBinWidth(i)
                l.DrawLine(x_low, 1, x_high, 1)
                h_dr.Draw('sameE') #redraw it to bring it to front
                
                canvas.dnd.append([pad, h_dr, h_tot, g_up, g_down])

    canvas.Draw()
    return canvas

In [ ]:
f = ur.open(glob(outdir + '/higgsCombine{}.FitDiagnostics.mH120.root'.format(card_name))[0])
c, d, u, _ = f['limit']['limit'].array()
print 'r = {:.1f} +{:.1f}/-{:.1f} %'.format(100*c, 100*(u-c), 100*(c-d))

N_B2mu = f['limit']['trackedParam_N_B2mu'].array()[0]
print 'N_B2mu = {:.0f}'.format(N_B2mu)
N_B2DstHc = f['limit']['trackedParam_N_B2DstHc'].array()[0]
print 'N_B2DstHc = {:.0f}'.format(N_B2DstHc)
N_B2Dstst = f['limit']['trackedParam_N_B2Dstst'].array()[0]
print 'N_B2Dstst = {:.0f}'.format(N_B2Dstst)

# Get post-fit shapes
f = rt.TFile(glob(outdir + '/fitDiagnostics{}.root'.format(card_name))[0], 'READ')
fd = f.shapes_fit_s

print '\n'
histo_postfit = {}
for cat, h_dic in histo.iteritems():
    histo_postfit[cat] = {}
    for n, h in h_dic.iteritems():
        if '__' in n:
            continue
        h_post = h.Clone(h.GetName() + '_postfit')
        if 'data' in n:
            h_fit = fd.Get(cat+'/total')
            h_data = h.Clone(h.GetName() + '_data')
            for i in range(1, h_post.GetNbinsX()+1):
                h_post.SetBinContent(i, h_fit.GetBinContent(i))
                h_post.SetBinError(i, h_fit.GetBinError(i))     
            
            histo_postfit[cat]['total'] = h_post
            histo_postfit[cat][n] = h_data
        else:
            h_fit = fd.Get(cat+'/'+n)
            for i in range(1, h_post.GetNbinsX()+1):
                h_post.SetBinContent(i, h_fit.GetBinContent(i))
                h_post.SetBinError(i, h_fit.GetBinError(i)) 

            histo_postfit[cat][n] = h_post
            
# c_out = plot_gridVarQ2(CMS_lumi, binning, histo_postfit, scale_dic={}, draw_pulls=True, pulls_ylim=[0.9, 1.1])

CMS_lumi.integrated_lumi = ''
histNorm = {}
for k, hdic in histo_postfit.iteritems():
    if not k in histNorm.keys():
        histNorm[k] = {}
    for kk, h in hdic.iteritems():
        hAux = h.Clone()
        hAux.Scale(1./float(N_data))
        histNorm[k][kk] = hAux

c = plot_gridVarQ2s(CMS_lumi, binning, histNorm, scale_dic={}, min_y=1, logy=False, draw_pulls=True, iPad_legend=3)

In [ ]:
c_out.SaveAs('gridPlot_test.png')

### Run likelyhood scan

In [ ]:
def getUncertainty(name, verbose=True):
    f = ur.open(name)
    r_arr = f['limit']['r'].array()
    nll_arr = f['limit']['deltaNLL'].array()
    c = r_arr[0]
    r_u = r_arr[r_arr>r_arr[0]]
    nll_u = nll_arr[r_arr>r_arr[0]]
    f_u = interp1d(nll_u, r_u, 'quadratic')
    u = f_u(0.5)
    r_l = r_arr[r_arr<r_arr[0]]
    nll_l = nll_arr[r_arr<r_arr[0]]
    f_l = interp1d(nll_l, r_l, 'quadratic')
    l = f_l(0.5)
    if verbose:
        print '----------------------------------'
        print 'r = {:.2f} +{:.2f}/-{:.2f} %'.format(100*c, 100*(u-c), 100*(c-l))
        print 'Sigma = {:.3f}'.format((u-l)*0.5)
        print '----------------------------------\n'
    return c, c-l, u-c, (u-l)*0.5

In [ ]:
cmd = 'combine'
cmd += ' -M MultiDimFit'
cmd += ' --algo grid --points=100'
cmd += ' --robustFit 1'
cmd += ' -d ' + card_location.replace('.txt', '.root')
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --rMin={:.4f} --rMax={:.4f}'.format(c - 5*(c-d), c + 5*(u-c))
cmd += ' -n {}_nominal'.format(card_name)
cmd += ' --verbose -1'
print cmd
os.system(cmd)
cmd = 'plot1DScan.py higgsCombine{}_nominal.MultiDimFit.mH120.root'.format(card_name)
cmd += '; mv scan.png scan_nominal.png'
os.system(cmd)
res_nominal = getUncertainty('higgsCombine{}_nominal.MultiDimFit.mH120.root'.format(card_name))
display(Image(filename='scan_nominal.png'))

### Uncertainy breakdown by group

In [ ]:
cmd = 'combine'
cmd += ' -M MultiDimFit'
cmd += ' --algo none'
cmd += ' --cminDefaultMinimizerStrategy=2'
cmd += ' --robustFit 1'
cmd += ' -d ' + card_location.replace('.txt', '.root')
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --setParameters r=0.1'
cmd += ' --rMin=0 --rMax=1'
cmd += ' -n {}_bestfit'.format(card_name)
cmd += ' --saveWorkspace'
cmd += ' --verbose -1'
print cmd
os.system(cmd);

In [ ]:
# Statistical uncertainty
cmd = 'combine'
cmd += ' -M MultiDimFit'
cmd += ' --algo grid --points=100'
cmd += ' --cminDefaultMinimizerStrategy=2'
cmd += ' --robustFit 1'
cmd += ' -d higgsCombine{}_bestfit.MultiDimFit.mH120.root'.format(card_name)
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --rMin={:.4f} --rMax={:.4f}'.format(c - 5*(c-d), c + 5*(u-c))
cmd += ' -n {}_stat'.format(card_name)
cmd += ' --snapshotName MultiDimFit'
cmd += ' --freezeParameters allConstrainedNuisances'
cmd += ' --verbose -1'
print cmd
os.system(cmd);
res_stat = getUncertainty('higgsCombine{}_stat.MultiDimFit.mH120.root'.format(card_name))
cmd = 'plot1DScan.py higgsCombine{}_nominal.MultiDimFit.mH120.root'.format(card_name)
cmd += ' --others "higgsCombine{}_stat.MultiDimFit.mH120.root:Freeze all:2"'.format(card_name)
cmd += ' --breakdown syst,stat'
cmd += '; mv scan.png scan_stat.png'
print cmd
os.system(cmd)
display(Image(filename='scan_stat.png'))

In [ ]:
# MC Statistics
cmd = 'combine'
cmd += ' -M MultiDimFit'
cmd += ' --algo grid --points=100'
cmd += ' --cminDefaultMinimizerStrategy=2'
cmd += ' --robustFit 1'
cmd += ' -d higgsCombine{}_bestfit.MultiDimFit.mH120.root'.format(card_name)
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --rMin={:.4f} --rMax={:.4f}'.format(c - 5*(c-d), c + 5*(u-c))
cmd += ' -n {}_MCstat'.format(card_name)
cmd += ' --snapshotName MultiDimFit'
cmd += ' --freezeNuisanceGroups=autoMCStats'
cmd += ' --verbose -1'
print cmd
os.system(cmd);
res_MCstat = getUncertainty('higgsCombine{}_MCstat.MultiDimFit.mH120.root'.format(card_name))
cmd = 'plot1DScan.py higgsCombine{}_nominal.MultiDimFit.mH120.root'.format(card_name)
cmd += ' --others'
cmd += ' "higgsCombine{}_MCstat.MultiDimFit.mH120.root:Freeze MC stat:4"'.format(card_name)
cmd += ' "higgsCombine{}_stat.MultiDimFit.mH120.root:Freeze all:2"'.format(card_name)
cmd += ' --breakdown MCstat,syst,stat'
cmd += '; mv scan.png scan_MCstat.png'
print cmd
os.system(cmd)
display(Image(filename='scan_MCstat.png'))

In [ ]:
os.system('mv higgsCombine*.root ' + outdir + '/')
os.system('mv scan* ' + outdir + '/')
os.system('mv combine_logger.out ' + outdir + '/');

### Run the pull/impact plots

Fit first the POI

In [ ]:
cmd = 'combineTool.py -M Impacts --doInitialFit -m 120'
cmd += ' --robustFit 1'
cmd += ' -d ' + card_location.replace('.txt', '.root')
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --setParameters r={:.2f}'.format(rawR_exp)
cmd += ' --setParameterRanges r=0.001,1'
cmd += ' -n {}'.format(card_name)
cmd += ' --out ' + outdir
cmd += ' --verbose 0'
os.system(cmd)

 Perform a similar scan for each nuisance parameter

In [ ]:
cmd = 'combineTool.py -M Impacts --doFits -m 120'
cmd += ' --robustFit 1'
cmd += ' --parallel 8'
cmd += ' -d ' + card_location.replace('.txt', '.root')
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' --X-rtd MINIMIZER_analytic'
# cmd += ' --setParameters r={:.2f}'.format(rawR_exp)
# cmd += ' --setParameterRanges r=0.001,1'
cmd += ' -n {}'.format(card_name)
cmd += ' --out ' + outdir
cmd += ' --verbose 0'
print cmd
os.system(cmd)

Collect the results

In [ ]:
cmd = 'combineTool.py -M Impacts -o impacts.json -m 120'
cmd += ' -d ' + card_location.replace('.txt', '.root')
cmd += ' -n {}'.format(card_name)
print cmd
os.system(cmd);

In [ ]:
rename = {
'B0pT': 'B_{0} p_{T} spectrum',
'N_B2mu':'N_{B#rightarrow D*#mu#nu}',
'N_B2DstHc':'N_{B#rightarrow D*H_{c}}',
'N_B2Dstst':'N_{B#rightarrow D**#mu#nu}',
'B2DstCLNR0':'R_{0} (CLN B#rightarrow D*)',
'B2DstCLNR1':'R_{1} (CLN B#rightarrow D*)',
'B2DstCLNR2':'R_{2} (CLN B#rightarrow D*)',
'B2DstCLNRhoSq':'#rho^{2} (CLN B#rightarrow D*)'
}
json.dump(rename, open('rename.json', 'w'))

cmd = 'plotImpacts.py -i impacts.json -o impacts -t rename.json'
os.system(cmd)
IFrame("impacts.pdf", width=900, height=600)

In [ ]:
os.system('mv *.root {}/'.format(outdir))
os.system('mv impacts.* {}/'.format(outdir))
os.system('mv rename.json {}/'.format(outdir))
os.system('mv combine_logger.out {}/'.format(outdir));

### Run the Goodness of fit test

Run the observed test stat value

In [ ]:
cmd = 'combine'
cmd += ' -M GoodnessOfFit'
cmd += ' -d ' + card_location
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' -n Obs'                                    # Just the output name
cmd += ' -t 0'                                      # Don't run toys
cmd += ' -s -1'                                     # Random seed
cmd += ' --toysNoSystematics --algo=saturated'
# cmd += ' --toysFrequentist'
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --setParameters r=0.1'
cmd += ' --setParameterRanges r=0.001,1'
cmd += ' --trackParameters N_B2mu'
cmd += ' --plots'
cmd += ' --verbose 9'

In [ ]:
print cmd
os.system(cmd)

Run the test stat toy distribution

In [ ]:
cmd = 'combine'
cmd += ' -M GoodnessOfFit'
cmd += ' -d ' + card_location
cmd += ' -D ' + histo[histo.keys()[0]]['data'].GetName()
cmd += ' -n Toys'                                   # Just the output name
cmd += ' -t 300'                                    # Number of toys to run
cmd += ' -s -1'                                     # Random seed
cmd += ' --toysNoSystematics --algo=saturated'
# cmd += ' --toysFrequentist'
# cmd += '--expectSignal=0'                           # Depending on the hypothesis to test. If none, r is fluctruated
cmd += ' --X-rtd MINIMIZER_analytic'
cmd += ' --setParameters r=0.1'
cmd += ' --setParameterRanges r=0.001,1'
cmd += ' --trackParameters N_B2mu'
cmd += ' --plots'
cmd += ' --verbose 1'

In [ ]:
print cmd
os.system(cmd)

Combine them to get the p-value

In [ ]:
name_obs = glob('higgsCombineObs.GoodnessOfFit.*.root')[0]
# name_obs = glob(outdir+'/higgsCombineObs.GoodnessOfFit.*.root')[0]
name_toys = glob('higgsCombineToys.GoodnessOfFit.*.root')[0]
# name_toys = glob(outdir+'/higgsCombineToys.GoodnessOfFit.*.root')[0]

In [ ]:
f = ur.open(name_obs)
s_obs = f['limit']['limit'].array()[0]

f = ur.open(name_toys)
s_toys = f['limit']['limit'].array()

content, center, _ = plt.hist(s_toys, label='Toys')
plt.plot([s_obs, s_obs], [0, np.max(content)], 'm--', label='Observed')
plt.legend(loc='best')

p_val = np.sum(s_toys > s_obs)/float(s_toys.shape[0])
print 'p-value: {:.1f}%'.format(100*p_val)
if p_val < 0.01: print p_val

In [ ]:
# cmd = 'combineTool.py'
# cmd += ' -M CollectGoodnessOfFit'
# cmd += ' --mass 120'
# cmd += ' -o gof.json'
# cmd += ' --input {} {}'.format(name_obs, name_toys)
# print cmd
# os.system(cmd)
# os.system('plotGof.py gof.json -o gof --mass 120.0')

In [ ]:
os.system('mv *.root {}/'.format(outdir))
os.system('mv *.dot {}/'.format(outdir))
os.system('mv *.out {}/'.format(outdir))